In [9]:
library(INLA)
library(maptools)
library(spdep)

In [10]:
#read in shapefile and data
county.map = readShapePoly('CountyBoundary/cb_2018_us_county_500k.shp',IDvar="GEOID")

Warning message:
“shapelib support is provided by GDAL through the sf and terra packages among others”


In [11]:
suic<-read.csv("suicide_0515.csv",header=TRUE)

In [12]:
head(suic)

,X,county,fips,year,Deaths,Population,SuicideMortalityRate
,<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>
1,117,"Autauga County, AL",1001,2005,NA,NA,NA
2,117,"Autauga County, AL",1001,2006,NA,NA,NA
3,117,"Autauga County, AL",1001,2007,NA,NA,NA
4,117,"Autauga County, AL",1001,2008,NA,NA,NA
5,117,"Autauga County, AL",1001,2009,NA,NA,NA
6,117,"Autauga County, AL",1001,2010,12,54571,21.9897


In [13]:
#create spatial data frame
polys<-SpatialPolygonsDataFrame(county.map,data=as.data.frame(county.map),match.ID=TRUE)

#obtain lat long coordinates
coords<-coordinates(polys)
polys$x<-coords[,1]
polys$y<-coords[,2]

#create adjacency matrix, neighbors list - here using Delaunay Triangulation
triang<-tri2nb(coords, row.names=NULL)
neib<-nb2WB(triang)
#calculate sum of number of neighbors

neib$sumnb<-sum(neib$num)
#how many neighbors for each county?

summary(neib$num)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  3.000   5.000   6.000   5.991   7.000  16.000 

In [14]:
set.seed(1234)
inla.geobugs2inla(neib$adj, neib$num, graph.file="suicides_map")
countyid<-rep(1:3233,each=11) #number of counties
countyid2<- countyid #number of counties for second random effect
resid<-rep(1:35563) #number of county-year observations
year<-rep(1:11,len=3233) #year variable

[1] "suicides_map"

In [15]:
head(suic)

,X,county,fips,year,Deaths,Population,SuicideMortalityRate
,<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>
1,117,"Autauga County, AL",1001,2005,NA,NA,NA
2,117,"Autauga County, AL",1001,2006,NA,NA,NA
3,117,"Autauga County, AL",1001,2007,NA,NA,NA
4,117,"Autauga County, AL",1001,2008,NA,NA,NA
5,117,"Autauga County, AL",1001,2009,NA,NA,NA
6,117,"Autauga County, AL",1001,2010,12,54571,21.9897


In [16]:
numerator<-suic$Deaths
denominator<-suic$Population
data<-data.frame(numerator, denominator, countyid, countyid2, resid, year)

In [17]:
formula7<-numerator~1+f(countyid,model="iid")+
f(countyid2,model="besag",graph="suicides_map")+f(year,model="rw1")+f(resid,model="iid")
result7<-inla(formula7,family="Binomial",Ntrials=denominator,data=data, verbose=TRUE, control.compute=list(dic=TRUE,cpo=TRUE))

In [18]:
#get fit statistics
result7$dic$dic;result7$dic$p.eff
# Extract the linear predictor
linear_predictor <- result7$summary.linear.predictor$mean
suicide_estimates <- (exp(linear_predictor) / (1 + exp(linear_predictor)))*100000
data$SuicideEstimates <- suicide_estimates

[1] 56964.24

[1] 2183.414

In [19]:
data

numerator,denominator,countyid,countyid2,resid,year,SuicideEstimates
<dbl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>
NA,NA,1,1,1,1,19.89822
NA,NA,1,1,2,2,19.87076
NA,NA,1,1,3,3,20.03600
NA,NA,1,1,4,4,20.16286
NA,NA,1,1,5,5,20.29169
12,54571,1,1,6,6,20.51253
12,55267,1,1,7,7,20.36298
12,55514,1,1,8,8,20.17170
11,55246,1,1,9,9,20.00505


In [20]:
suic

X,county,fips,year,Deaths,Population,SuicideMortalityRate
<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>
117,"Autauga County, AL",1001,2005,NA,NA,NA
117,"Autauga County, AL",1001,2006,NA,NA,NA
117,"Autauga County, AL",1001,2007,NA,NA,NA
117,"Autauga County, AL",1001,2008,NA,NA,NA
117,"Autauga County, AL",1001,2009,NA,NA,NA
117,"Autauga County, AL",1001,2010,12,54571,21.989701
117,"Autauga County, AL",1001,2011,12,55267,21.712776
117,"Autauga County, AL",1001,2012,12,55514,21.616169
117,"Autauga County, AL",1001,2013,11,55246,19.910944


In [21]:
# Concatenate dataframes horizontally
result <- cbind(suic, data)
head(result)

,X,county,fips,year,Deaths,Population,SuicideMortalityRate,numerator,denominator,countyid,countyid2,resid,year,SuicideEstimates
,<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>
1,117,"Autauga County, AL",1001,2005,NA,NA,NA,NA,NA,1,1,1,1,19.89822
2,117,"Autauga County, AL",1001,2006,NA,NA,NA,NA,NA,1,1,2,2,19.87076
3,117,"Autauga County, AL",1001,2007,NA,NA,NA,NA,NA,1,1,3,3,20.03600
4,117,"Autauga County, AL",1001,2008,NA,NA,NA,NA,NA,1,1,4,4,20.16286
5,117,"Autauga County, AL",1001,2009,NA,NA,NA,NA,NA,1,1,5,5,20.29169
6,117,"Autauga County, AL",1001,2010,12,54571,21.9897,12,54571,1,1,6,6,20.51253


In [22]:
result

X,county,fips,year,Deaths,Population,SuicideMortalityRate,numerator,denominator,countyid,countyid2,resid,year,SuicideEstimates
<int>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>
117,"Autauga County, AL",1001,2005,NA,NA,NA,NA,NA,1,1,1,1,19.89822
117,"Autauga County, AL",1001,2006,NA,NA,NA,NA,NA,1,1,2,2,19.87076
117,"Autauga County, AL",1001,2007,NA,NA,NA,NA,NA,1,1,3,3,20.03600
117,"Autauga County, AL",1001,2008,NA,NA,NA,NA,NA,1,1,4,4,20.16286
117,"Autauga County, AL",1001,2009,NA,NA,NA,NA,NA,1,1,5,5,20.29169
117,"Autauga County, AL",1001,2010,12,54571,21.989701,12,54571,1,1,6,6,20.51253
117,"Autauga County, AL",1001,2011,12,55267,21.712776,12,55267,1,1,7,7,20.36298
117,"Autauga County, AL",1001,2012,12,55514,21.616169,12,55514,1,1,8,8,20.17170
117,"Autauga County, AL",1001,2013,11,55246,19.910944,11,55246,1,1,9,9,20.00505


In [23]:
write.csv(result, file = "suicide_0515_imputed.csv", row.names = FALSE)